**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 3.5MB/s 


In [0]:
# !pip install -U tensorFlow

In [0]:
# !pip uninstall tf-nightly-gpu

In [2]:
import keras 
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

import tensorflow as tf
from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Dropout

import warnings
warnings.simplefilter('ignore')
tf.test.gpu_device_name()

Using TensorFlow backend.


'/device:GPU:0'

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self, s, train=False):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act is reprsenting a $\epsilon$ greedy version of the learned policy with a reinforcement learning method. In fact, it's a way of selecting random actions with uniform distribution from a set of available actions with a certain probability in order to push the agent to explore the environemment and not just acting with the learned policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
      
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=21 # set small when debugging
epochs_test=11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

* The array **position** enables to have the state of the environemment, it  trakes the movement of the agent (By setting values of the cells where he passed to 1).   
* The array **board**  is a abstract representation of the board. It enables to access rewards from the cells that are either a cheese + 0.5, a normal cell +0 or a poison -1 .



## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        
        action = np.random.randint(0, self.n_action, size=1)[0]
        
        return action 

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.0/9.0. Average score (1.0)
Win/lose count 10.5/19.0. Average score (-3.75)
Win/lose count 18.5/9.0. Average score (0.6666666666666666)
Win/lose count 10.5/18.0. Average score (-1.375)
Win/lose count 7.0/7.0. Average score (-1.1)
Win/lose count 11.5/18.0. Average score (-2.0)
Win/lose count 17.0/17.0. Average score (-1.7142857142857142)
Win/lose count 7.0/12.0. Average score (-2.125)
Win/lose count 10.5/10.0. Average score (-1.8333333333333333)
Win/lose count 12.0/20.0. Average score (-2.45)
Win/lose count 10.5/19.0. Average score (-3.0)
Final score: -3.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




From the Defition of the Q function:

\begin{equation}
\begin{aligned}
Q^{\pi}(s, a) &=E_{p^{\pi}}\left[\sum_{t = 0}^{\infty} \gamma^{t} r\left(s_{t}, a_{t}\right) | s_{0}=s, a_{0}=a\right]\\
&= E_{p^{\pi}}\left[r\left(s, a\right) + \sum_{t = 1}^{\infty} \gamma^{t} r\left(s_{t}, a_{t}\right) | s_{0}=s, a_{0}=a\right]\\
&= E_{p^{\pi}}\left[r\left(s, a\right) + \gamma \sum_{t = 0}^{\infty} \gamma^{t-1} r\left(s_{t}, a_{t}\right) | s_{0}=s, a_{0}=a\right]\\
&= E_{p^{\pi}}\left[r\left(s, a\right) + \sum_{t = 0}^{\infty} \gamma^{t} r\left(s_{t}, a_{t}\right) | s_{0}=s, a_{0}=a\right]\\
&=\sum_{s^{\prime}} \mathcal{P}_{s s^{\prime}}^{a}\left[r\left(s, a\right)+\gamma \mathbb{E}_{p^{\pi}}\left[\sum_{k=0}^{\infty} \gamma^{k} r\left(s^{\prime}_{t}\right) | s_{t+1}=s^{\prime}\right]\right]\\
&= \sum_{s^{\prime}} \mathcal{P}_{s s^{\prime}}^{a}\left[r\left(s, a\right)+\gamma \sum_{a^{\prime}} \pi\left(s^{\prime}, a^{\prime}\right) Q^{\pi}\left(s^{\prime}, a^{\prime}\right)\right]\\
&=E_{\left(s^{\prime}, a^{\prime}\right) \sim p(. | s, a)}\left[r(s, a)+\gamma Q^{\pi}\left(s^{\prime}, a^{\prime}\right)\right]
\end{aligned}
\end{equation}

Thus, the optimal Q function is:
\begin{equation}
\begin{aligned}
Q^{*}(s, a)&=\max _{\pi} Q^{\pi}(s, a)\\
&= \max _{\pi} E_{p^{\pi}}\left[r\left(s, a\right) + \gamma  \sum_{s^{\prime}} \mathcal{P}_{s s^{\prime}}^{a} Q^{\pi}\left(s^{\prime}, a^{\prime}\right)| s_{0}=s, a_{0}=a\right]\\
&= E_{p^{\pi}}\left[r\left(s, a\right) + \gamma  \sum_{s^{\prime}} \max _{\pi} \mathcal{P}_{s s^{\prime}}^{a} Q^{\pi}\left(s^{\prime}, a^{\prime}\right)| s_{0}=s, a_{0}=a\right]
\end{aligned}
\end{equation}
And since :
\begin{equation}
\pi^{*}(s)=\arg \max _{a} Q^{*}(s, a) \quad \forall s \in \mathbb{S}
\end{equation}
Thus 
\begin{equation}
\begin{aligned}
Q^{*}(s, a)&= E_{s^{\prime} \sim \pi^{*}(. | s, a)}\left[r(s, a)+\gamma \max _{a^{\prime}} Q^{*}\left(s^{\prime}, a^{\prime}\right)\right]
\end{aligned}
\end{equation}

And since we have $E_{s^{\prime} \sim \pi^{*}(. | s, a)}\left[r(s, a)+\gamma \max _{a^{\prime}} Q^{*}\left(s^{\prime}, a^{\prime}\right)\right]$.
 which is equivalent to $E_{s^{\prime} \sim \pi^{*}(. | s, a)}\left[r(s, a)+\gamma \max _{a^{\prime}} Q^{*}\left(s^{\prime}, a^{\prime}\right)-Q^{*}(s, a) \right] =0 $. Then a natural objective to minimize is the difference between the two quantities ( or also , the square 'L2-norm').  
 Thus $\mathcal{L}(\theta)=E_{s^{\prime} \sim \pi^{*}(\cdot | s, a)}\left\|r+\gamma \max _{a^{\prime}} Q\left(s^{\prime}, a^{\prime}, \theta\right)-Q(s, a, \theta)\right\|^{2}$ is a loss that will ensure the convergence of a certain neural network model predicting the Q-function to verify the previous equality (under some other hypothesis). 


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
      self.memory.append(m)
      if len(self.memory) > self.max_memory :
        del self.memory[0]
        
    def random_access(self):
        random_m = self.memory[np.random.randint(0, len(self.memory))]
        return random_m

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch, verbose=True, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    Final_average_score = np.zeros(epoch)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            print('Experience of epoch {} saved as : {}'.format(e, prefix+str(e)))

        # Update stats
        score += win-lose
        Final_average_score[e] = win-lose
        if verbose :
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        else :
          if e % 10 == 0:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
    return np.mean(Final_average_score)

__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.


In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()

            input_states[i] = s_            
            if game_over_:
                # If we reach the terminal state (Game is over) target_Q(state,action) is just  the reward associated to the one observed
                target_q[i, a_] = r_
            else:
                # If we didn't reach the terminal state, target_Q(state,action) is the reward associated to the action performed + max target_Q(n_state,action')
                max_predicted_value = np.max(self.model.predict(np.expand_dims(n_s_, axis=0)))
                target_q[i, a_] = r_ + self.discount * max_predicted_value

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model

        model = Sequential(layers = [Flatten(input_shape=(5, 5, self.n_state)),
                                    Dense(100, activation="relu"), 
                                    Dropout(0.2), 
                                    Dense(100, activation="relu"), 
                                    Dropout(0.2),
                                    Dense(40, activation="relu"),
                                    Dropout(0.2),
                                    Dense(4)])
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
for batch_size in [32, 64, 128]:
  print('batch_size = ', batch_size)
  env = Environment(grid_size=size, max_time=T, temperature=0.3)
  agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = batch_size)
  avg_score = train(agent, env, epochs_train, verbose=False, prefix='fc_train_bs'+str(batch_size)+'_')
  print('Average Score is : {:.3f}'.format(avg_score))
  print('*'*10)
# HTML(display_videos('fc_train20.mp4'))

batch_size =  32
Experience of epoch 0 saved as : fc_train_bs32_0
Epoch 000/021 | Loss 0.0024 | Win/lose count 0.5/1.0 (-0.5)
Experience of epoch 10 saved as : fc_train_bs32_10
Epoch 010/021 | Loss 0.0177 | Win/lose count 3.0/10.0 (-7.0)
Experience of epoch 20 saved as : fc_train_bs32_20
Epoch 020/021 | Loss 0.0042 | Win/lose count 7.0/10.0 (-3.0)
Average Score is : -5.571
**********
batch_size =  64
Experience of epoch 0 saved as : fc_train_bs64_0
Epoch 000/021 | Loss 0.0077 | Win/lose count 4.5/9.0 (-4.5)
Experience of epoch 10 saved as : fc_train_bs64_10
Epoch 010/021 | Loss 0.0156 | Win/lose count 5.0/8.0 (-3.0)
Experience of epoch 20 saved as : fc_train_bs64_20
Epoch 020/021 | Loss 0.0089 | Win/lose count 2.0/6.0 (-4.0)
Average Score is : -14.048
**********
batch_size =  128
Experience of epoch 0 saved as : fc_train_bs128_0
Epoch 000/021 | Loss 0.0119 | Win/lose count 6.5/7.0 (-0.5)
Experience of epoch 10 saved as : fc_train_bs128_10
Epoch 010/021 | Loss 0.0091 | Win/lose count 3.

In [0]:
for m_size in [100, 1000, 2000]:
  print('Memory size = ', m_size)
  env = Environment(grid_size=size, max_time=T, temperature=0.3)
  agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=m_size, batch_size = 32)
  avg_score = train(agent, env, epochs_train, verbose=False, prefix='fc_train_'+str(m_size)+'_')
  print('Average Score is : {:.3f}'.format(avg_score))
  print('*'*10)
# HTML(display_videos('fc_train20.mp4'))

Memory size =  100
Experience of epoch 0 saved as : fc_train_100_0
Epoch 000/021 | Loss 0.0042 | Win/lose count 4.0/4.0 (0.0)
Experience of epoch 10 saved as : fc_train_100_10
Epoch 010/021 | Loss 0.0113 | Win/lose count 8.0/12.0 (-4.0)
Experience of epoch 20 saved as : fc_train_100_20
Epoch 020/021 | Loss 0.0002 | Win/lose count 3.5/4.0 (-0.5)
Average Score is : -9.619
**********
Memory size =  1000
Experience of epoch 0 saved as : fc_train_1000_0
Epoch 000/021 | Loss 0.0192 | Win/lose count 5.0/5.0 (0.0)
Experience of epoch 10 saved as : fc_train_1000_10
Epoch 010/021 | Loss 0.0094 | Win/lose count 3.5/2.0 (1.5)
Experience of epoch 20 saved as : fc_train_1000_20
Epoch 020/021 | Loss 0.0222 | Win/lose count 9.5/15.0 (-5.5)
Average Score is : -4.310
**********
Memory size =  2000
Experience of epoch 0 saved as : fc_train_2000_0
Epoch 000/021 | Loss 0.0037 | Win/lose count 4.0/9.0 (-5.0)
Experience of epoch 10 saved as : fc_train_2000_10
Epoch 010/021 | Loss 0.0120 | Win/lose count 6.5/

In [0]:
HTML(display_videos('fc_train_1000_20.mp4'))

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
avg_score = train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train20.mp4'))

Experience of epoch 0 saved as : fc_train0
Epoch 000/021 | Loss 0.0117 | Win/lose count 6.5/7.0 (-0.5)
Epoch 001/021 | Loss 0.0094 | Win/lose count 6.0/7.0 (-1.0)
Epoch 002/021 | Loss 0.0060 | Win/lose count 5.5/4.0 (1.5)
Epoch 003/021 | Loss 0.0020 | Win/lose count 8.0/2.0 (6.0)
Epoch 004/021 | Loss 0.0107 | Win/lose count 4.0/4.0 (0.0)
Epoch 005/021 | Loss 0.0105 | Win/lose count 0.5/4.0 (-3.5)
Epoch 006/021 | Loss 0.0122 | Win/lose count 5.0/5.0 (0.0)
Epoch 007/021 | Loss 0.0285 | Win/lose count 4.0/4.0 (0.0)
Epoch 008/021 | Loss 0.0080 | Win/lose count 7.5/4.0 (3.5)
Epoch 009/021 | Loss 0.0061 | Win/lose count 5.5/3.0 (2.5)
Experience of epoch 10 saved as : fc_train10
Epoch 010/021 | Loss 0.0042 | Win/lose count 2.5/3.0 (-0.5)
Epoch 011/021 | Loss 0.0139 | Win/lose count 5.0/5.0 (0.0)
Epoch 012/021 | Loss 0.0045 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/021 | Loss 0.0019 | Win/lose count 6.0/4.0 (2.0)
Epoch 014/021 | Loss 0.0214 | Win/lose count 3.0/2.0 (1.0)
Epoch 015/021 | Loss 0.

After running multiple experiments with different parameters and architectures, we noticed that : 

*   The choice of $\epsilon$ is very crucial for a good performance. Indeed, with a greedy policy (very low value of $\epsilon$, the agent exploits too much and thus ignore other options that could be better which leads to poor performance, the same for a large value of $\epsilon$ that leads to random actions and therefore too much exploration that causes increasing total loss and neglecting optimal actions. Values between 0.1 and 0.4 seems to give better results. (Another efficient method is to use a decaying epsilon).
*   For the learning rate, values in the range of 0.1 are more suitable for training since they are not too low ( which causes a slow convergence) and not too high ( undesirable divergent behavior).
*   Also, the training phase has a different behavior when using different values of the memory size. The default value 2000 is enabling to have great average score.
*   When increasing the batch size, the training gets slower but the performance of training do not increase as well.


Perhaps, One can run multiple experiences to confirm those results and to build a confidence interval on the impact of each parameter. Also, the different parameters are interacting , so we need to try different combinaisons to retrieve good parameters.



***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN

        model = Sequential(layers = [Conv2D(50, (2,2), input_shape=(5,5,self.n_state), activation='relu'),
                                    Conv2D(100, (2,2), activation='relu'), 
                                    Dropout(0.2), 
                                    Flatten(),
                                    Dense(4)])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [19]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train20.mp4'))

Experience of epoch 0 saved as : cnn_train0
Epoch 000/021 | Loss 0.0110 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/021 | Loss 0.0188 | Win/lose count 3.5/10.0 (-6.5)
Epoch 002/021 | Loss 0.0198 | Win/lose count 4.0/5.0 (-1.0)
Epoch 003/021 | Loss 0.0094 | Win/lose count 4.5/8.0 (-3.5)
Epoch 004/021 | Loss 0.0102 | Win/lose count 9.0/4.0 (5.0)
Epoch 005/021 | Loss 0.0065 | Win/lose count 3.5/1.0 (2.5)
Epoch 006/021 | Loss 0.0043 | Win/lose count 3.5/2.0 (1.5)
Epoch 007/021 | Loss 0.0105 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/021 | Loss 0.0080 | Win/lose count 8.0/1.0 (7.0)
Epoch 009/021 | Loss 0.0116 | Win/lose count 4.0/0 (4.0)
Experience of epoch 10 saved as : cnn_train10
Epoch 010/021 | Loss 0.0048 | Win/lose count 4.0/1.0 (3.0)
Epoch 011/021 | Loss 0.0023 | Win/lose count 2.0/8.0 (-6.0)
Epoch 012/021 | Loss 0.0086 | Win/lose count 5.0/2.0 (3.0)
Epoch 013/021 | Loss 0.0011 | Win/lose count 4.0/2.0 (2.0)
Epoch 014/021 | Loss 0.0024 | Win/lose count 6.0/2.0 (4.0)
Epoch 015/021 | Loss

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 0/0. Average score (0.0)
Win/lose count 1.5/0. Average score (0.75)
Win/lose count 1.0/0. Average score (0.8333333333333334)
Win/lose count 2.0/0. Average score (1.125)
Win/lose count 0.5/1.0. Average score (0.8)
Win/lose count 0/0. Average score (0.6666666666666666)
Win/lose count 0.5/2.0. Average score (0.35714285714285715)
Win/lose count 0/1.0. Average score (0.1875)
Win/lose count 1.0/2.0. Average score (0.05555555555555555)
Win/lose count 2.0/1.0. Average score (0.15)
Win/lose count 1.0/1.0. Average score (0.13636363636363635)
Final score: 0.13636363636363635
Test of the FC
Win/lose count 0.5/2.0. Average score (-1.5)
Win/lose count 1.5/7.0. Average score (-3.5)
Win/lose count 0.5/4.0. Average score (-3.5)
Win/lose count 3.5/5.0. Average score (-3.0)
Win/lose count 2.5/2.0. Average score (-2.3)
Win/lose count 2.0/1.0. Average score (-1.75)
Win/lose count 0/3.0. Average score (-1.9285714285714286)
Win/lose count 2.0/4.0. Average score (-1.9375)
Win/lo

In [0]:
HTML(display_videos('cnn_test10.mp4'))

In [0]:
HTML(display_videos('fc_test10.mp4'))

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.15)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 0.5/2.0. Average score (-0.75)
Win/lose count 1.0/0. Average score (-0.16666666666666666)
Win/lose count 1.0/0. Average score (0.125)
Win/lose count 0/0. Average score (0.1)
Win/lose count 0/0. Average score (0.08333333333333333)
Win/lose count 1.5/5.0. Average score (-0.42857142857142855)
Win/lose count 0.5/0. Average score (-0.3125)
Win/lose count 2.0/1.0. Average score (-0.16666666666666666)
Win/lose count 0.5/2.0. Average score (-0.3)
Win/lose count 0/0. Average score (-0.2727272727272727)
Final score: -0.2727272727272727
Test of the FC
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0/2.0. Average score (-1.5)
Win/lose count 1.0/2.0. Average score (-1.3333333333333333)
Win/lose count 0.5/0. Average score (-0.875)
Win/lose count 0/0. Average score (-0.7)
Win/lose count 0.5/0. Average score (-0.5)
Win/lose count 0/0. Average score (-0.42857142857142855)
Win/lose count 0.5/0. Average score (-0.3125)


In [0]:
env = Environment(grid_size=size, max_time=T,temperature=0.85)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 3.0/0. Average score (3.0)
Win/lose count 1.5/2.0. Average score (1.25)
Win/lose count 1.0/0. Average score (1.1666666666666667)
Win/lose count 1.0/1.0. Average score (0.875)
Win/lose count 0.5/1.0. Average score (0.6)
Win/lose count 1.0/0. Average score (0.6666666666666666)
Win/lose count 5.0/1.0. Average score (1.1428571428571428)
Win/lose count 5.0/0. Average score (1.625)
Win/lose count 0/1.0. Average score (1.3333333333333333)
Win/lose count 0.5/0. Average score (1.25)
Win/lose count 5.5/1.0. Average score (1.5454545454545454)
Final score: 1.5454545454545454
Test of the FC
Win/lose count 4.0/3.0. Average score (1.0)
Win/lose count 3.5/1.0. Average score (1.75)
Win/lose count 4.5/1.0. Average score (2.3333333333333335)
Win/lose count 2.5/1.0. Average score (2.125)
Win/lose count 2.5/0. Average score (2.2)
Win/lose count 3.0/0. Average score (2.3333333333333335)
Win/lose count 4.0/1.0. Average score (2.4285714285714284)
Win/lose count 0.5/0. Average sc

Testing the two types of networks on grids with different temperatures we remark that Convolutional neural networs yields better resutls that Fully connected network since it manages to achieve good positive rewards; Also, Low values of temperature impacts highly the agent score since it appears that he do not manage to retrieve bonus states contrary to environements with high temperature values where the agent get high positive rewards in average. However, It's important to note that when agent get all the time stuck in states with no promising rewards which explains their lack of exploration.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,initial_epsilon=0.9, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    agent.set_epsilon(initial_epsilon)
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state, train=True)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Updating the epsilon of the agent (Decreasing ϵ -greedy exploration)
        updated_epsilon = agent.epsilon**2
        agent.set_epsilon(updated_epsilon)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            print('Experience of epoch {} saved as : {}'.format(e, prefix+str(e)))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.malus_position = np.zeros_like(self.board)

    def act(self, action , train=False):
      
      self.get_frame(int(self.t))
      self.position = np.zeros((self.grid_size, self.grid_size))
      self.position[0:2,:]= -1
      self.position[:,0:2] = -1
      self.position[-2:, :] = -1
      self.position[-2:, :] = -1

      self.position[self.x, self.y] = 1
      if action == 0:
          if self.x == self.grid_size-3:
              self.x = self.x-1
          else:
              self.x = self.x + 1
      elif action == 1:
          if self.x == 2:
              self.x = self.x+1
          else:
              self.x = self.x-1
      elif action == 2:
          if self.y == self.grid_size - 3:
              self.y = self.y - 1
          else:
              self.y = self.y + 1
      elif action == 3:
          if self.y == 2:
              self.y = self.y + 1
          else:
              self.y = self.y - 1
      else:
          RuntimeError('Error: action not recognized')

      self.t = self.t + 1
    
    # ## In Environment exploring:
    # # You will have to change n_state to 3 because you will use one more layer!
      reward = 0
      if train:
          # a negative reward of 0.1 of the position was visited
          reward = self.malus_position[self.x, self.y]
      # Mark the current position as visited
      self.malus_position[self.x, self.y] = -0.1

      reward += self.board[self.x, self.y]
      self.board[self.x, self.y] = 0
      game_over = self.t > self.max_time
      # 3 "feature" states instead of 2 for this environnement
      state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                              self.board.reshape(self.grid_size, self.grid_size,1),
                              self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
      state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

      return state, reward, game_over

    def reset(self):

        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.malus_position[self.x,self.y] = -0.1
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                              self.board.reshape(self.grid_size, self.grid_size,1),
                              self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state



In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, epochs_train, initial_epsilon=0.9, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore20.mp4'))



Experience of epoch 0 saved as : cnn_train_explore0
Epoch 000/021 | Loss 0.0135 | Win/lose count 10.5/28.200000000000067 (-17.700000000000067)
Epoch 001/021 | Loss 0.0128 | Win/lose count 8.0/23.10000000000003 (-15.10000000000003)
Epoch 002/021 | Loss 0.0155 | Win/lose count 14.0/22.200000000000063 (-8.200000000000063)
Epoch 003/021 | Loss 0.0171 | Win/lose count 15.0/18.1 (-3.1000000000000014)
Epoch 004/021 | Loss 0.0193 | Win/lose count 14.5/20.700000000000003 (-6.200000000000003)
Epoch 005/021 | Loss 0.0171 | Win/lose count 11.0/19.600000000000016 (-8.600000000000016)
Epoch 006/021 | Loss 0.0188 | Win/lose count 12.5/16.899999999999974 (-4.399999999999974)
Epoch 007/021 | Loss 0.0255 | Win/lose count 8.0/19.90000000000002 (-11.90000000000002)
Epoch 008/021 | Loss 0.0117 | Win/lose count 8.5/16.09999999999996 (-7.599999999999959)
Epoch 009/021 | Loss 0.0082 | Win/lose count 8.0/17.399999999999977 (-9.399999999999977)
Experience of epoch 10 saved as : cnn_train_explore10
Epoch 010/0

In [23]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 13.0/3.0. Average score (10.0)
Win/lose count 7.0/0. Average score (8.5)
Win/lose count 14.5/2.0. Average score (9.833333333333334)
Win/lose count 29.5/4.0. Average score (13.75)
Win/lose count 17.0/2.0. Average score (14.0)
Win/lose count 13.0/1.0. Average score (13.666666666666666)
Win/lose count 13.0/0. Average score (13.571428571428571)
Win/lose count 16.0/0. Average score (13.875)
Win/lose count 9.5/1.0. Average score (13.277777777777779)
Win/lose count 5.5/3.0. Average score (12.2)
Win/lose count 11.5/3.0. Average score (11.863636363636363)
Final score: 11.863636363636363


In order to boost the performance of the agent and enhance it's exploration capability, we used the two proposed ameliorations by corporating (a decreasing  $\epsilon$-greedy exploration and a new state that describes if a cell has been visited or not). Thus, when testing them on the new agent, it appears that they helped increasing the rewards and the exploration on the test phase. Contrary to previous experiences, the agent is able to collect large positive rewards (>15) from the grid. However, when the agent is stuck in one of the corners with no positive rewards, he don't manage to escape from it, but he showed a great ability to dodge negative rewards. 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

For this question, we will train an FC agent to mimic the behavior of the expert DQN CNN agent trained before. For this purpose, we will use the same train function as before. but with the use of actions proposed by the expert DQN and a direct reinforcement of the FC agent.  
The FC agent will have only 2 states as before and 


***

In [0]:
def train_mimic(expert_agent, mimic_agent, env,epoch, verbose=True, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The expert agent performs an action as if it was tested 
            action = expert_agent.act(state, train=False)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            # We consider that the agent will have a number of states equal to 2 'as before'.
            # for that we modify prev_state and state in order to not take account of the malus position loss 
            loss = mimic_agent.reinforce(prev_state[:,:,1:], state[:,:,1:],  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            print('Experience of epoch {} saved as : {}'.format(e, prefix+str(e)))

        # Update stats
        score += win-lose
        if verbose :
          print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        else :
          if e % 10 == 0:
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                .format(e, epoch, loss, win, lose, win-lose))
        mimic_agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    


In [37]:
expert_DQN = agent
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train_mimic(expert_DQN, agent_fc, env, epoch=20, prefix='fc_train_mimic')
HTML(display_videos('fc_train_mimic20.mp4'))

Experience of epoch 0 saved as : fc_train_mimic0
Epoch 000/020 | Loss 0.0090 | Win/lose count 8.0/0 (8.0)
Epoch 001/020 | Loss 0.0024 | Win/lose count 11.5/2.0 (9.5)
Epoch 002/020 | Loss 0.0059 | Win/lose count 1.5/0 (1.5)
Epoch 003/020 | Loss 0.0078 | Win/lose count 9.5/1.0 (8.5)
Epoch 004/020 | Loss 0.0080 | Win/lose count 6.0/0 (6.0)
Epoch 005/020 | Loss 0.0043 | Win/lose count 10.0/1.0 (9.0)
Epoch 006/020 | Loss 0.0035 | Win/lose count 15.0/0 (15.0)
Epoch 007/020 | Loss 0.0049 | Win/lose count 12.5/1.0 (11.5)
Epoch 008/020 | Loss 0.0081 | Win/lose count 3.5/0 (3.5)
Epoch 009/020 | Loss 0.0053 | Win/lose count 8.5/0 (8.5)
Experience of epoch 10 saved as : fc_train_mimic10
Epoch 010/020 | Loss 0.0030 | Win/lose count 11.5/1.0 (10.5)
Epoch 011/020 | Loss 0.0124 | Win/lose count 5.5/1.0 (4.5)
Epoch 012/020 | Loss 0.0142 | Win/lose count 14.0/1.0 (13.0)
Epoch 013/020 | Loss 0.0045 | Win/lose count 3.5/0 (3.5)
Epoch 014/020 | Loss 0.0066 | Win/lose count 17.5/1.0 (16.5)
Epoch 015/020 | L

We will Evaluate the FC agent:

In [40]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_mimic_test')

Test of the FC
Win/lose count 1.0/2.0. Average score (-1.0)
Win/lose count 2.0/4.0. Average score (-1.5)
Win/lose count 0/0. Average score (-1.0)
Win/lose count 0/1.0. Average score (-1.0)
Win/lose count 0.5/0. Average score (-0.7)
Win/lose count 0.5/1.0. Average score (-0.6666666666666666)
Win/lose count 0.5/3.0. Average score (-0.9285714285714286)
Win/lose count 0/2.0. Average score (-1.0625)
Win/lose count 0/0. Average score (-0.9444444444444444)
Win/lose count 3.0/3.0. Average score (-0.85)
Win/lose count 2.5/3.0. Average score (-0.8181818181818182)
Final score: -0.8181818181818182


In [41]:
HTML(display_videos('fc_mimic_test10.mp4'))

Surprisingly, the FC agent is not performing well and he's just juggling between 2 states at a certain time. he don't manage to get large positive rewards on the long term as for the expert DQN agent.  
We will train another network to see if this behavior changes.  

In [0]:
class DQN_FC_2(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC_2, self).__init__( *args,**kwargs)
        
        # NN Model

        model = Sequential(layers = [Flatten(input_shape=(5, 5, self.n_state)),
                                    Dense(50, activation="relu"),
                                    Dropout(0.3),
                                    Dense(4)])
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [48]:
expert_DQN = agent
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC_2(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train_mimic(expert_DQN, agent_fc, env, epoch=31, prefix='fc2_train_mimic')
HTML(display_videos('fc2_train_mimic30.mp4'))

Experience of epoch 0 saved as : fc2_train_mimic0
Epoch 000/031 | Loss 0.0042 | Win/lose count 3.5/0 (3.5)
Epoch 001/031 | Loss 0.0166 | Win/lose count 12.5/1.0 (11.5)
Epoch 002/031 | Loss 0.0218 | Win/lose count 8.0/1.0 (7.0)
Epoch 003/031 | Loss 0.0167 | Win/lose count 7.5/0 (7.5)
Epoch 004/031 | Loss 0.0124 | Win/lose count 7.5/0 (7.5)
Epoch 005/031 | Loss 0.0100 | Win/lose count 11.5/1.0 (10.5)
Epoch 006/031 | Loss 0.0112 | Win/lose count 6.5/0 (6.5)
Epoch 007/031 | Loss 0.0032 | Win/lose count 10.5/2.0 (8.5)
Epoch 008/031 | Loss 0.0150 | Win/lose count 14.0/4.0 (10.0)
Epoch 009/031 | Loss 0.0168 | Win/lose count 12.0/2.0 (10.0)
Experience of epoch 10 saved as : fc2_train_mimic10
Epoch 010/031 | Loss 0.0081 | Win/lose count 7.5/0 (7.5)
Epoch 011/031 | Loss 0.0088 | Win/lose count 6.5/1.0 (5.5)
Epoch 012/031 | Loss 0.0043 | Win/lose count 8.0/0 (8.0)
Epoch 013/031 | Loss 0.0097 | Win/lose count 10.5/1.0 (9.5)
Epoch 014/031 | Loss 0.0093 | Win/lose count 7.5/1.0 (6.5)
Epoch 015/031 |

In [56]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc2_mimic_test')
HTML(display_videos('fc2_mimic_test10.mp4'))

Test of the FC
Win/lose count 5.0/1.0. Average score (4.0)
Win/lose count 5.5/2.0. Average score (3.75)
Win/lose count 2.5/0. Average score (3.3333333333333335)
Win/lose count 4.5/1.0. Average score (3.375)
Win/lose count 0.5/0. Average score (2.8)
Win/lose count 7.0/1.0. Average score (3.3333333333333335)
Win/lose count 3.0/0. Average score (3.2857142857142856)
Win/lose count 5.5/0. Average score (3.5625)
Win/lose count 0.5/0. Average score (3.2222222222222223)
Win/lose count 2.0/0. Average score (3.1)
Win/lose count 9.5/5.0. Average score (3.227272727272727)
Final score: 3.227272727272727


In fact, using a smaller network with only one layer, the agent is performing well on average compared to it's previous performance with a normal training. However, he's not yet able to retrieve larger rewards as the expert DQN (It can be due to the additional state the expert is given??). 